# TensorFlow 2 Quickstart

## Libraries

In [1]:
import numpy as np
import tensorflow as tf

## Data Preparation

In [2]:

mnist_db = tf.keras.datasets.mnist

In [3]:

# Load the TF pregenarated MNIST dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
print(x_train[198])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 101 222 159 113 114  12   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  57 163 226 249 252 252 252 253 228  97   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  76 234 252 253 252 252 220 252 253 252 239  66   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 197 252 252 225  99  84  37  84 146 249 252 180  13
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  48 227 252 173   0   0   0   0   0   0 225 25

In [5]:
print(y_train[198])

3


In [6]:

# Normalise training data by dividing by largest possible value (255)
x_train, x_test = x_train / 255.0, x_test / 255.0

## Specify-Compile-Fit Workflow

In [7]:

# Build a tf.keras.Sequential model by stacking layers
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=x_train.shape[1:]),
  tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [8]:

# model returns a vector of log-odds scores, one for each class
predictions = model(x_train[:1]).numpy()
print("Predictions:", predictions)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Predictions: [[ 0.17284195 -0.01639419  0.07244249 -0.11882917  0.07131827  0.20710023
   0.10186651  0.14150506  0.19258296 -0.08318442]]


In [9]:

# The tf.nn.softmax function calibrates logits to probabilities for each class
print("Probabilities for each class: \n", tf.nn.softmax(predictions).numpy())

Probabilities for each class: 
 [[0.10974436 0.09082344 0.09926113 0.08198059 0.09914961 0.11356915
  0.10222519 0.10635863 0.11193234 0.08495548]]


In [10]:

# Define loss function
# Sparse Categorical Cross Entropy loss returns a scalar loss for each example
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:

# Configure model's optimizer, loss and evaluation metric (among others)
# Compile the configured model in the same step
model.compile(optimizer='adam',
              loss=loss_function,
              metrics=['accuracy'])

In [12]:

# Fit the model to data
# This optimisation step adjusts the model's parameters (weights) and miminimises the loss
model.fit(x_train, y_train, epochs=3)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 5s 81us/sample - loss: 0.5818 - accuracy: 0.8207
Epoch 2/3
60000/60000 [==============================] - 4s 65us/sample - loss: 0.3054 - accuracy: 0.9133
Epoch 3/3
60000/60000 [==============================] - 4s 63us/sample - loss: 0.2609 - accuracy: 0.9259


## Model Evaluation

In [13]:

# Evaluate the model on a validation/test set
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 0s - loss: 0.1171 - accuracy: 0.9481


[0.18367004224471747, 0.9481]

In [24]:
print(x_test[0])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

In [23]:

# To calibrate the model and return a probability, wrap the trained model and attach a softmax layer
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

print(probability_model(x_test[:1]))

tf.Tensor(
[[4.86681802e-06 4.61770014e-06 9.99458611e-01 4.68435872e-04
  5.68079095e-09 4.49653817e-05 1.87424382e-06 1.14786935e-05
  5.17282115e-06 2.63142175e-10]], shape=(1, 10), dtype=float32)
